In [1]:
from torch.utils.data import Dataset, DataLoader
from libs.Normalization import MinMaxNormalizer, BasicNormalizer, ImageNetNormalizer
from libs.Dataset_val import PatchDataGenerator
import matplotlib.pyplot as plt
import numpy as np

/sw/external/python/anaconda3_gpu/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torch.optim import Adam
import wandb

# Function to calculate loss (replace with your desired loss function)
def dice_loss(pred, target):
    smooth = 1.
    intersection = (pred * target).sum(dim=(2,3)) 
    loss = 1 - ((2. * intersection + smooth) / (pred.sum(dim=(2,3)) + target.sum(dim=(2,3)) + smooth))
    return loss.mean()

def train_model(model, train_loader, val_loader, epochs, learning_rate):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    optimizer = Adam(model.parameters(), lr=learning_rate)
    criterion = dice_loss 

    for epoch in range(epochs):
        running_loss = 0.0
        for i, data in enumerate(train_loader, 0):
            inputs, labels = data[0].to(device), data[1].to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()  

            if i % 10 == 9: 
                print(f'Epoch [{epoch+1}/{epochs}], Step [{i+1}/{len(train_loader)}], Loss: {running_loss / 10:.3f}')
                wandb.log({"train_loss": running_loss / 10})
                running_loss = 0.0

        # Validation (optional)
        if val_loader:
            with torch.no_grad():
                val_loss = 0.0
                for _, data in enumerate(val_loader, 0):
                    inputs, labels = data[0].to(device), data[1].to(device)
                    outputs = model(inputs)
                    val_loss += criterion(outputs, labels).item()
                print(f'Epoch [{epoch+1}], Val Loss: {val_loss / len(val_loader):.3f}')
                wandb.log({"val_loss": val_loss / len(val_loader)})


In [3]:
import torch
import torch.optim as optim
from torch.utils.data import DataLoader
from torch import nn
from models.unet import UNet
from models.unetTransformer import U_Transformer

# Hyperparameters
batch_size = 16
learning_rate = 1e-3
num_epochs = 150


In [4]:
# Create dataset and dataloader
traing_generator = PatchDataGenerator(patch_size=256, norm_type='basic', hue_factor=1, augment=True)
train_data = DataLoader(traing_generator, batch_size=batch_size, shuffle=True, num_workers=4)

# Create dataset and dataloader
validation_generator = PatchDataGenerator(data_dir="/projects/bcxi/nathanj/commonPatchData/validation", patch_size=256, norm_type='basic', hue_factor=1, augment=True)
validation_data = DataLoader(validation_generator, batch_size=batch_size, shuffle=True, num_workers=4)


Setup complete. Number of maps loaded: 169


*****Training Initialization ****
 data dir : /projects/bcxi/nathanj/commonPatchData/256/15
*********************************
Setup complete. Number of maps loaded: 40
*****Training Initialization ****
 data dir : /projects/bcxi/nathanj/commonPatchData/validation/256/15
*********************************


In [5]:
import os
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'max_split_size_mb:512'

In [6]:
# Function to calculate loss (replace with your desired loss function)
def dice_loss(pred, target):
    smooth = 1.
    intersection = (pred * target).sum(dim=(2,3)) 
    loss = 1 - ((2. * intersection + smooth) / (pred.sum(dim=(2,3)) + target.sum(dim=(2,3)) + smooth))
    return loss.mean()

def train_model(model, train_loader, val_loader, epochs, learning_rate):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    optimizer = Adam(model.parameters(), lr=learning_rate)
    criterion = dice_loss 

    for epoch in range(epochs):
        running_loss = 0.0
        for i, data in enumerate(train_loader, 0):
            inputs, labels = data[0].to(device).float(), data[1].to(device).float()
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()  

            if i % 10 == 9: 
                print(f'Epoch [{epoch+1}/{epochs}], Step [{i+1}/{len(train_loader)}], Loss: {running_loss / 10:.3f}')
#                 wandb.log({"train_loss": running_loss / 10})
                running_loss = 0.0
    
            # Clear unused memory
            torch.cuda.empty_cache()

        # Validation (optional)
        if val_loader:
            with torch.no_grad():
                val_loss = 0.0
                for _, data in enumerate(val_loader, 0):
                    inputs, labels = data[0].to(device).float(), data[1].to(device).float()
                    outputs = model(inputs)
                    val_loss += criterion(outputs, labels).item()
                print(f'Epoch [{epoch+1}], Val Loss: {val_loss / len(val_loader):.3f}')
#                 wandb.log({"val_loss": val_loss / len(val_loader)})

In [ ]:
# Initialize model, loss function, and optimizer
model = U_Transformer(in_channels=6, classes=1)
train_model(model, train_data, validation_data, num_epochs, learning_rate)

Epoch [1/150], Step [10/18199], Loss: 0.874
Epoch [1/150], Step [20/18199], Loss: 0.795
Epoch [1/150], Step [30/18199], Loss: 0.734
Epoch [1/150], Step [40/18199], Loss: 0.730
Epoch [1/150], Step [50/18199], Loss: 0.757
Epoch [1/150], Step [60/18199], Loss: 0.677
Epoch [1/150], Step [70/18199], Loss: 0.724
Epoch [1/150], Step [80/18199], Loss: 0.695
Epoch [1/150], Step [90/18199], Loss: 0.714
Epoch [1/150], Step [100/18199], Loss: 0.780
Epoch [1/150], Step [110/18199], Loss: 0.774
Epoch [1/150], Step [120/18199], Loss: 0.764
Epoch [1/150], Step [130/18199], Loss: 0.756
Epoch [1/150], Step [140/18199], Loss: 0.756
Epoch [1/150], Step [150/18199], Loss: 0.725
Epoch [1/150], Step [160/18199], Loss: 0.756
Epoch [1/150], Step [170/18199], Loss: 0.718
Epoch [1/150], Step [180/18199], Loss: 0.780
Epoch [1/150], Step [190/18199], Loss: 0.692
Epoch [1/150], Step [200/18199], Loss: 0.708
Epoch [1/150], Step [210/18199], Loss: 0.688
Epoch [1/150], Step [220/18199], Loss: 0.719
Epoch [1/150], Step

In [2]:
import cv2
import numpy as np
from PIL import Image
import time
# Increase the maximum pixel limit
Image.MAX_IMAGE_PIXELS = None

# Function to read a TIFF file and convert to HSV
def convert_rgb_to_hsv(tiff_file_path):
    # Open the TIFF file using PIL
    with Image.open(tiff_file_path) as img:
        # Convert PIL image to NumPy array (RGB format)
        rgb_array = np.array(img)
        
        # Convert RGB to HSV using OpenCV
        hsv_array = cv2.cvtColor(rgb_array, cv2.COLOR_RGB2HSV)
        
        return hsv_array

# Specify the path to your TIFF file
tiff_file_path = '/projects/bbym/shared/data/sortedDatasets/training/images/MT_Kalispell.tif'

# Time the conversion process
start_time = time.time()
hsv_array = convert_rgb_to_hsv(tiff_file_path)
end_time = time.time()

# Calculate the elapsed time
elapsed_time = end_time - start_time

# Print the resulting HSV array and elapsed time
print("HSV Array:", hsv_array)
print(f"Elapsed Time: {elapsed_time:.6f} seconds")

HSV Array: [[[  0   0 255]
  [  0   0 255]
  [  0   0 255]
  ...
  [  0   0 255]
  [  0   0 255]
  [  0   0 255]]

 [[  0   0 255]
  [  0   0 255]
  [  0   0 255]
  ...
  [  0   0 255]
  [  0   0 255]
  [  0   0 255]]

 [[  0   0 255]
  [  0   0 255]
  [  0   0 255]
  ...
  [  0   0 255]
  [  0   0 255]
  [  0   0 255]]

 ...

 [[  0   0 255]
  [  0   0 255]
  [  0   0 255]
  ...
  [  0   0 255]
  [  0   0 255]
  [  0   0 255]]

 [[  0   0 255]
  [  0   0 255]
  [  0   0 255]
  ...
  [  0   0 255]
  [  0   0 255]
  [  0   0 255]]

 [[  0   0 255]
  [  0   0 255]
  [  0   0 255]
  ...
  [  0   0 255]
  [  0   0 255]
  [  0   0 255]]]
Elapsed Time: 5.849921 seconds


In [4]:
!pip install pytorch-lightning

Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 802.2/802.2 kB 2.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 779.1/779.1 MB 8.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.2/176.2 MB 14.1 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.1/168.1 MB 11.7 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 868.8/868.8 kB 4.2 MB/s eta 0:00:00:00:01
  Attempting uninstall: triton
    Found existing installation: triton 2.1.0
    Uninstalling triton-2.1.0:
      Successfully uninstalled triton-2.1.0
  Attempting uninstall: nvidia-nccl-cu12
    Found existing installation: nvidia-nccl-cu12 2.18.1
    Uninstalling nvidia-nccl-cu12-2.18.1:
      Successfully uninstalled nvidia-nccl-cu12-2.18.1


In [1]:
import os

import torch
from torch import nn, optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.optim import Adam

from pytorch_lightning import LightningModule, Trainer
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.loggers import WandbLogger
from torchmetrics.classification import BinaryAccuracy

import matplotlib.pyplot as plt
import numpy as np

from models.unet import UNet
from models.unetTransformer import U_Transformer

from libs.Normalization import MinMaxNormalizer, BasicNormalizer, ImageNetNormalizer
from libs.Dataset_val import PatchDataGenerator

os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'max_split_size_mb:512'

class LitUTransformer(LightningModule):
    def __init__(self, model, learning_rate=1e-3):
        super(LitUTransformer, self).__init__()
        self.model = model
        self.learning_rate = learning_rate
        self.criterion = self.dice_loss
        self.accuracy = BinaryAccuracy()

    def dice_loss(self, pred, target):
        smooth = 1.
        pred = torch.sigmoid(pred)
        intersection = (pred * target).sum(dim=(2,3))
        loss = 1 - ((2. * intersection + smooth) / (pred.sum(dim=(2,3)) + target.sum(dim=(2,3)) + smooth))
        return loss.mean()

    def forward(self, x):
        return self.model(x)

    def training_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = self.criterion(y_hat, y)
        acc = self.accuracy((y_hat > 0.5).float(), y)
        self.log('train_loss', loss)
        self.log('train_acc', acc, prog_bar=True)
        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = self.criterion(y_hat, y)
        acc = self.accuracy((y_hat > 0.5).float(), y)
        self.log('val_loss', loss, prog_bar=True)
        self.log('val_acc', acc, prog_bar=True)

    def test_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = self.criterion(y_hat, y)
        acc = self.accuracy((y_hat > 0.5).float(), y)
        self.log('test_loss', loss)
        self.log('test_acc', acc)

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=self.learning_rate)
        return optimizer

# Hyperparameters
batch_size = 16
learning_rate = 1e-3
num_epochs = 150



/sw/external/python/anaconda3_gpu/lib/python3.9/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: libtorch_cuda_cu.so: cannot open shared object file: No such file or directory
  warn(f"Failed to load image Python extension: {e}")
/sw/external/python/anaconda3_gpu/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
# Create dataset and dataloader
train_generator = PatchDataGenerator(data_dir="/projects/bcxi/nathanj/commonPatchData/", patch_size=256, overlap=32, norm_type='basic', hue_factor=1, valid_patch_rate=1, augment=True)
train_data = DataLoader(train_generator, batch_size=batch_size, shuffle=True, num_workers=4)

validation_generator = PatchDataGenerator(data_dir="/projects/bcxi/nathanj/commonPatchData/validation", patch_size=256, overlap=32, norm_type='basic', hue_factor=1, valid_patch_rate=1, augment=True)
validation_data = DataLoader(validation_generator, batch_size=batch_size, shuffle=True, num_workers=4)


Setup complete. Number of maps loaded: 168


*****Training Initialization ****
 data dir : /projects/bcxi/nathanj/commonPatchData/256/32
*********************************
Setup complete. Number of maps loaded: 40
*****Training Initialization ****
 data dir : /projects/bcxi/nathanj/commonPatchData/validation/256/32
*********************************


In [3]:
# Initialize model
model = U_Transformer(in_channels=6, classes=1)

# Initialize Lightning model
lit_model = LitUTransformer(model, learning_rate)

# Initialize Wandb logger
wandb_logger = WandbLogger(project='U_Transformer_Segmentation')

# Model checkpoint callback
checkpoint_callback = ModelCheckpoint(
    monitor='val_loss',
    dirpath='checkpoints',
    filename='u-transformer-{epoch:02d}-{val_loss:.2f}',
    save_top_k=3,
    mode='min',
)

# Trainer
trainer = Trainer(
    max_epochs=num_epochs,
    callbacks=[checkpoint_callback],
    logger=wandb_logger
)

# Training
trainer.fit(lit_model, train_data, validation_data)


Trainer will use only 1 of 2 GPUs because it is running inside an interactive / notebook environment. You may try to set `Trainer(devices=2)` but please note that multi-GPU inside interactive / notebook environments is considered experimental and unstable. Your mileage may vary.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA A100 80GB PCIe') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
wandb: Currently logged in as: 9bombs. Use `wandb login --relogin` to force relogin


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]

  | Name     | Type           | Params
--------------------------------------------
0 | model    | U_Transformer  | 11.7 M
1 | accuracy | BinaryAccuracy | 0     
--------------------------------------------
11.7 M    Trainable params
0         Non-trainable params
11.7 M    Total params
46.718    Total estimated model params size (MB)
SLURM auto-requeueing enabled. Setting signal handlers.


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/u/nathanj/.local/lib/python3.9/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:492: Your `val_dataloader`'s sampler has shuffling enabled, it is strongly recommended that you turn shuffling off for val/test dataloaders.


Training: |          | 0/? [00:00<?, ?it/s]

/u/nathanj/.local/lib/python3.9/site-packages/pytorch_lightning/trainer/call.py:54: Detected KeyboardInterrupt, attempting graceful shutdown...
